In [ ]:
import mnist_help as mh
import numpy as np
#import NeuralNetwork as nn
import matplotlib.pyplot as plt
import time
from numba import jit


class Timer:
    @classmethod
    def __now(cls):
        return time.time()

    def __init__(self):
        self.start_time = self.last_lap = self.__now()
        pass

    def lap(self, out=True):
        now = self.__now()
        rs = now - self.last_lap
        self.last_lap = now
        if out:
            print(rs)
            pass
        return rs

    def time(self):
        print(self.__now() - self.start_time)

    def reset(self):
        self.start_time = self.last_lap = self.__now()
        pass

    pass

def show(image):
    plt.imshow(image, cmap='Greys_r')  # 显示图片
    plt.axis('off')  # 不显示坐标轴
    plt.show()
    return

#images, labels, testimages, testlabels = mh.load_mnist()
#images = images. reshape (-1,28,28,1)
#image = images[0:200]

In [ ]:
import numpy as np
from numba import jit
filters=np.random.rand(20,3,3,1)
filters[:] = np.array([-0.125,-0.125,-0.125,-0.125,1,-0.125,-0.125,-0.125,-0.125]).reshape(3,3,1)

def _conv_kernel(data, filters, bias, result, stride1, stride2):
    n2, d2, w2, h2 = filters.shape
    n3, d3, w3, h3 = result.shape
    one_filter = filters.reshape(n2, -1).T
    for ww in range(w3):
        for hh in range(h3):
            d = (data[:, :, ww * stride1:ww * stride1 + w2, hh * stride2:hh * stride2 + h2]).reshape(n3, -1)
            result[:, :, ww, hh] = d @ one_filter + bias
    return


@jit(nopython=True)
def _conv_kernel1(data, K, b, rs, s1, s2):
    n2, d2, w2, h2 = K.shape
    n3, d3, w3, h3 = rs.shape
    for nn in range(n3):
        for dd in range(d3):
            for ww in range(w3):
                for hh in range(h3):
                    d = data[nn, :, ww * s1:ww * s1 + w2, hh * s2:hh * s2 + h2]
                    kn = K[dd]
                    rs[nn, dd, ww, hh] = (d * kn).sum()
    return


@jit(nopython=True,nogil = True)
def _conv_kernel2(data, K, b, rs, s1, s2):
    n2, d2, w2, h2 = K.shape
    n3, d3, w3, h3 = rs.shape
    for nn in range(n3):
        for dd in range(d3):
            for ww in range(w3):
                for hh in range(h3):
                    rs[nn, dd, ww, hh] = 0
                    for kd in range(d2):
                        for kw in range(w2):
                            for kh in range(h2):
                                rs[nn, dd, ww, hh] += K[dd, kd, kw, kh] * data[nn, kd, ww * s1 + kw, hh * s2 + kh]
    return


def _conv(data, filters, bias, strides, pad, conv_kernel):
    """
    :param data: shape (n,d,w,h)
    :param filters:shape (n,d,w,h)
    """
    n1, d1, w1, h1 = data.shape
    n2, d2, w2, h2 = filters.shape
    w3 = (w1 - w2 + 2 * pad[0]) / strides[0] + 1
    h3 = (h1 - h2 + 2 * pad[1]) / strides[0] + 1
    n3, d3 = n1, n2
    assert w3 % 1 == 0 and h3 % 1 == 0 and d1 == d2
    w3, h3 = int(w3), int(h3)
    result = np.empty((n3, d3, w3, h3))
    pad_data = np.pad(data, ((0, 0), (0, 0), (pad[0], pad[0]), (pad[1], pad[1])), 'constant')
    conv_kernel(pad_data, filters, bias, result, strides[0], strides[1])
    return result


def conv(data, filters, bias, strider, pad, f):
    return _conv(data, filters, bias, (strider, strider), (pad, pad), f)


def _conv_gradient(x, filters, diff, strides, pad):
    n1, d1, w1, h1 = x.shape
    n2, d2, w2, h2 = filters.shape
    w3 = (w1 - w2 + 2 * pad[0]) / strides[0] + 1
    h3 = (h1 - h2 + 2 * pad[1]) / strides[0] + 1
    assert w3 % 1 == 0 and h3 % 1 == 0 and d1 == d2
    assert (n1, n2, int(w3), int(h3)) == diff.shape
    n3, d3, w3, h3 = diff.shape
    pad_data = np.pad(x, ((0, 0), (0, 0), (pad[0], pad[0]), (pad[1], pad[1])), 'constant')
    delta_filters = np.zeros_like(filters)
    delta_bias = np.zeros((1, n2))
    delta_filters_tmp = delta_filters.reshape(n2, -1)
    s1, s2 = strides
    for ww in range(w3):
        for hh in range(h3):
            d = (pad_data[:, :, ww * s1:ww * s1 + w2, hh * s2:hh * s2 + h2]).reshape(n3, -1)
            delta_filters_tmp += diff[:, :, ww, hh].T @ d
            delta_bias += diff[:, :, ww, hh].sum(axis=0, keepdims=True)
    return delta_filters, delta_bias


def conv_gradient(x, filters, diff, stride, pad): return _conv_gradient(x, filters, diff, (stride, stride), (pad, pad))


    
test = np.random.rand(200,3,28,28)
ff = np.random.rand(8,3,3,3)
lens=100
t = Timer ()
t.lap()
for i in range(lens):
    k1 = conv(test,ff,0,1,1,_conv_kernel)
t.lap()
# for i in range(lens):
#     k2 = conv(test,ff,0,1,1,_conv_kernel1)
t.lap()
for i in range(lens):
    k3 = conv(test,ff,0,1,1,_conv_kernel2)
t.lap()

print ((k1-k3).sum()<1e-8)
print ((k2 - k3).sum()<1e-8)

# print(_conv_gradient(test,ff,k1,(1,1),(1,1)))

#show(k1[4,:,:,0])#.reshape(28,28))

In [ ]:
import numpy as np
a = np.arange(2)
1<a

In [ ]:
import numpy as np
import numba as nb

a = np.arange(10000).reshape(100,100)
@nb.jit(nopython=True)
def sum1(a,b):
    x,y = a.shape
    rs=0
    for i in range(x):
        for k in range(y):
            rs+=a[i,k]*b[i,k]
    return rs
@nb.jit(nopython=True)
def sum2(a,b):
    return (a*b).sum()
help (np.zeros)
#%timeit sum1(a,a)
#%timeit (a*a).sum()
#%timeit sum2(a,a)

In [ ]:
import numba as nb
import math
#@nb.jit(nopython=True)
@nb.vectorize
def r(x):
    rr = x
    if x<0:
        rr *=0.01
    return rr

@nb.jit(nopython=True)
#@nb.vectorize(nopython=True)
def rr(x,w):
    qq=y
    if x<0:
        qq*=0.01
    return qq

@nb.vectorize(nopython=True)
def sigmod(x):return math.exp(x)

a = np.random.rand(1000,100,1000)
#print (a.dot(a).shape)
%timeit r(a)
%timeit np.maximum(0,a)

In [ ]:
#np.save('data/train1',images)
d1 = np.load('data/train1.npy')

In [4]:
import numpy as np
def prelu(x, r=0.05): return np.minimum(np.maximum(x * r, x),x*r+(1-r))

def prelu_back(y, diff, r=0): 
    tmp = np.where(y > 0, diff, diff * r)
    tmp = np.where(y < 1, tmp, tmp * r)
    return tmp

a = np.arange(10)*0.4-2
r = prelu_back(a,1)
print(a)
print(r)

[-2.  -1.6 -1.2 -0.8 -0.4  0.   0.4  0.8  1.2  1.6]
[0 0 0 0 0 0 1 1 0 0]
